# [5-3]트리의 앙상블

- 핵심 키워드
  - 앙상블 학습
  - 랜덤 포레스트
  - 엑스트라 트리
  - 그라디언트 부스팅

#### 정형 데이터와 비정형 데이터

- 정형 데이터(structed data)
: 가공된 데이터 ex] csv, excel, database

- 비정형 데이터(unstructed data)
: Ex] 사진, 영상 .. 등


정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 알고릐즘이  
 앙상블(ensemble learning) (결정 트리 기반으로 만들어짐)

## 랜덤포레스트(Random Forest)
- 앙상블 학습의 대표 알고리즘
- 안정적인 성능
- 결정트리르 랜덤하게 만들어 숲을 만듬(각 결정트리의 예측을 사용해 최종 예측)

부트스트랩 사용
: 데이터 세트에서 중복을 허용하여 데이터 샘플링하는 방식

[예시]
사이킷런의 RandomForestClassfier 와인 분류 모델에 적용

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [5]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)
#retain_train_score =True : 검증 점수뿐만 아니라 훈련 세트에 대한 점수도 같이 반환          #default =False
#n_jobs=-1: cpu 병렬로 사용

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


> 랜덤 포레스틑 결정 트리의 앙상블이기 때문에 DecisionTreeClassifier가 제공하는 중요한 매개 변수 모두 제공

In [3]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [4]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리(Extra Trees)
- 랜덤 포레스트와 비슷하게 동작
- 전체 특성 중 일부 특성을 랜덤하게 선택하여 노드 분할에 사용
- 랜더 포레스트와 차이점은 부트스트랩 샘플 사용 x
- 즉각 결정 트리를 만들 때 전체 훈련 세트를 사용

사이킷런에서 제공하는 ExtraTressClassifier 사용

In [7]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


> 보통 엑스트라 트리가 무작위성이 좀 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리 훈련 필요  
> 하지만 랜덤하게 노드를 분할하기 때문에 빠른 계산 속도가 장점

In [8]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅(Gradient boosting)
- 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법
- 그레이디언트 부스팅은 결정 트리르 계속 추가하면서 가장 낮은 곳을 찾아 이동(4장  경사 하강법과 같은 메커니즘)
- 결정 트리의 개수를 늘려도 과대적합에 매우 강함
- 학습률을 증가시키고 트리의 개수를 늘리면 성능 향상

GradientBoostingClassifier 기본적으로 깊이 3인 결정 트리 100개 사용

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [12]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [13]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 부스팅(Histogram-based Gradient Boosting)
: 먼저 특성을 256개 구간으로 나눔 -> 노드를 분할할 때 최적의 분할을 빠르게 찾을 수 있음

- 256개의 구간 중에서 하나를 떼어놓고 누락된 값을 위해서 사용 -> 입력에 누락된 특성이 있더라도 전처리 필요 X

트리의 개수를 지정하는데 max_estimators 대신 부스팅 반복 횟수를 지정하는 max_iter 사용

In [27]:
# 사이킷런 1.0 버전 아래에서는 다음 라인의 주석을 해제하고 실행하세요.

from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [20]:
from sklearn.inspection import permutation_importance

hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [21]:
result = permutation_importance(hgb, test_input, test_target, n_repeats=10,
                                random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.05969231 0.20238462 0.049     ]


In [22]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [24]:
from xgboost import XGBClassifier

#tree_method = 'hist' 사용하면 히스토르램 기반 그레디언트 부스팅 알고리즘 사용 가능
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9555033709953124 0.8799326275264677


#### LightGBM
- 히스토그램 기반 그레디언트 부스팅 라이브러리 made by MS
- 속도가 빠르고 최신 기술 많이 적용

In [26]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884
